In [1]:

# imports
from mewpy.germ import  FBA, single_gene_deletion, SRFBA, RFBA, isingle_gene_deletion, slim_pfba, fva, slim_fba
from mewpy.io import read_model, Reader, Engines
from cobra.io import read_sbml_model
from reframed import load_cbmodel


<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [4]:
cobra_model = read_sbml_model("/home/sebastiao_zoio/Documents/test_mewpy/mewpy/examples/models/germ/e_coli_core.xml")
reframed_model = load_cbmodel("/home/sebastiao_zoio/Documents/test_mewpy/mewpy/examples/models/germ/e_coli_core.xml", use_infinity=False)


cobra_reader = Reader(Engines.CobraModelEngine, cobra_model)
reframed_reader = Reader(Engines.ReframedModel, reframed_model)


gem_reader = Reader(Engines.MetabolicSBML, "/home/sebastiao_zoio/Documents/test_mewpy/mewpy/examples/models/germ/e_coli_core.xml")

trn_model = "/home/sebastiao_zoio/Documents/test_mewpy/mewpy/examples/models/germ/e_coli_core_trn.csv"

trn_reader = Reader(Engines.BooleanRegulatoryCSV,
                    trn_model,
                    sep=',',
                    id_col=0,
                    rule_col=2,
                    aliases_cols=[1],
                    header=0)

germ_model = read_model(trn_reader,cobra_reader)

#germ_model = read_model(reframed_reader, trn_reader)

old_gem_model = read_model(gem_reader, trn_reader)

In [34]:
df = single_gene_deletion(germ_model)


In [35]:
# Extract IDs and set them as the index
df['ids'] = df['ids'].str.strip('{}')  # Remove curly braces
df.set_index('ids', inplace=True)

# Rename columns to match desired format
df.columns = ['growth', 'status']

# Capitalize the 'status' column
df['status'] = df['status'].str.capitalize()

# Print the transformed DataFrame
print(df)

       growth   status
ids                   
NaN  0.873922  Optimal
NaN  0.873922  Optimal
NaN  0.873922  Optimal
NaN  0.796696  Optimal
NaN  0.870745  Optimal
..        ...      ...
NaN  0.873922  Optimal
NaN  0.873922  Optimal
NaN  0.873922  Optimal
NaN  0.873922  Optimal
NaN  0.873922  Optimal

[137 rows x 2 columns]


In [6]:
single_gene_deletion(old_gem_model)

,growth,status
b0351,0.873922,Optimal
b1241,0.873922,Optimal
s0001,0.211141,Optimal
b2296,0.873922,Optimal
b3115,0.873922,Optimal
...,...,...
b2464,0.873922,Optimal
b0008,0.873922,Optimal
b2935,0.873922,Optimal
b2465,0.873922,Optimal


In [6]:
SRFBA(old_gem_model).build().optimize( constraints={'ACONTa': (0.0, 0.0)})

Method,SRFBA
Model,Model e_coli_core_trn - E. coli core model - Orth et al 2010
Objective,Biomass_Ecoli_core
Objective value,0.8739215069684829
Status,optimal


In [39]:
opt_val = slim_fba(old_gem_model)

for rxn in old_gem_model.yield_reactions():
    constraint = {rxn.id: (0.0, 0.0)}
    c_opt_val = slim_fba(old_gem_model, constraints=constraint)
    if c_opt_val != opt_val:
        print((rxn.id, c_opt_val))

('ACONTa', 0.0)
('ACONTb', 0.0)
('AKGDH', 0.8583074080226883)
('ATPM', 0.9166474637510486)
('ATPS4r', 0.3742298749331098)
('Biomass_Ecoli_core', 0.0)
('CO2t', 0.4616696141601343)
('CS', 0.0)
('CYTBD', 0.2116629497353106)
('ENO', 0.0)
('EX_co2_e', 0.4616696141601343)
('EX_glc__D_e', 0.0)
('EX_h_e', 1.734723475976807e-15)
('EX_h2o_e', 0.383515449296783)
('EX_nh4_e', 0.0)
('EX_o2_e', 0.2116629497353106)
('EX_pi_e', -1.1102230246251565e-15)
('FBA', 0.7040369478590237)
('FRUpts2', 0.8739215069684302)
('FUM', 0.8142975075325303)
('G6PDH2r', 0.8638133095040007)
('GAPD', 0.0)
('GLCpts', 0.0)
('GLNS', 0.0)
('GLUDy', 0.8513885233462081)
('GND', 0.8638133095040007)
('H2Ot', 0.383515449296783)
('ICDHyr', 0.0)
('MDH', 0.8258192905969886)
('NADH16', 0.2116629497353106)
('NH4t', 0.0)
('O2t', 0.2116629497353106)
('PDH', 0.7966959254309567)
('PFK', 0.7040369478590237)
('PGI', 0.863159552208418)
('PGK', 0.0)
('PGL', 0.8638133095040007)
('PGM', 0.0)
('PIt2r', -1.1102230246251565e-15)
('PPC', 0.8707448062

In [43]:
slim_fba(old_gem_model, constraints={'ACONTa': (0.0, 0.0)})

0.0

In [47]:
cnstr = {'ACONTa': (0.0, 0.0)}

fba = FBA(old_gem_model).build()

fba.constraints

{'acald_c': <mewpy.germ.lp.linear_containers.ConstraintContainer at 0x7f34f0b7e670>,
 'coa_c': <mewpy.germ.lp.linear_containers.ConstraintContainer at 0x7f34f0b7ea90>,
 'nad_c': <mewpy.germ.lp.linear_containers.ConstraintContainer at 0x7f34f2665f40>,
 'accoa_c': <mewpy.germ.lp.linear_containers.ConstraintContainer at 0x7f34f2665f10>,
 'h_c': <mewpy.germ.lp.linear_containers.ConstraintContainer at 0x7f34f2665160>,
 'nadh_c': <mewpy.germ.lp.linear_containers.ConstraintContainer at 0x7f34f2665eb0>,
 'acald_e': <mewpy.germ.lp.linear_containers.ConstraintContainer at 0x7f34f2665d00>,
 'ac_c': <mewpy.germ.lp.linear_containers.ConstraintContainer at 0x7f34f2665700>,
 'atp_c': <mewpy.germ.lp.linear_containers.ConstraintContainer at 0x7f34f26655b0>,
 'actp_c': <mewpy.germ.lp.linear_containers.ConstraintContainer at 0x7f34f2665b80>,
 'adp_c': <mewpy.germ.lp.linear_containers.ConstraintContainer at 0x7f34f2665220>,
 'cit_c': <mewpy.germ.lp.linear_containers.ConstraintContainer at 0x7f34f2665a30>,

In [14]:
df = single_gene_deletion(old_gem_model)
filtered_df = df.loc[df['status'] == 'Infeasible']
filtered_df

,growth,status
b2415,0.0,Infeasible
b2416,0.0,Infeasible


In [17]:
old_gem_model.genes['b2416']

Identifier,b2416
Name,b2416
Aliases,"ptsI, b2416"
Model,e_coli_core_trn
Types,"gene, target"
Coefficients,"(0.0, 1.0)"
Active,True
Reactions,"FRUpts2, GLCpts"
Interaction,b2416 || 1 = 1
Regulators,


In [3]:
import time


In [4]:
trn_model = "/home/sebastiao_zoio/Documents/test_mewpy/mewpy/examples/models/germ/e_coli_core_trn.csv"

trn_reader = Reader(Engines.BooleanRegulatoryCSV,
                    trn_model,
                    sep=',',
                    id_col=0,
                    rule_col=2,
                    aliases_cols=[1],
                    header=0)

In [5]:
def get_new_times():
    start = time.process_time()
    reframed_model = read_sbml_model("/home/sebastiao_zoio/Documents/test_mewpy/mewpy/examples/models/germ/e_coli_core.xml")
    reframed_time = time.process_time() - start
    start = time.process_time()
    reframed_reader = Reader(Engines.CobraModelEngine, reframed_model)
    reader_time = time.process_time() - start
    start = time.process_time()
    #read_model(reframed_reader)
    read_model(reframed_reader, trn_reader)
    read_model_time = time.process_time() - start
    return reframed_time,reader_time, read_model_time

In [6]:
times = []

for e in range(50):
    times.append(get_new_times())

times
reframed_times = 0.0
reader_times = 0.0
read_model_times = 0.0
for t in times:
    reframed_times += t[0]
    reader_times += t[1]
    read_model_times += t[2]

n = len(times)
new_times_means = reframed_times/n, reader_times/n, read_model_times/n

In [7]:
new_times_means

(0.22775526887999972, 6.690497999951361e-05, 0.60762382406)

In [25]:
def get_old_times():
    """ start = time.process_time()
    cobra_model = read_sbml_model("/home/sebastiao_zoio/Documents/test_mewpy/mewpy/examples/models/germ/e_coli_core.xml")
    cobra_time = time.process_time() - start """
    start = time.process_time()
    gem_reader = Reader(Engines.MetabolicSBML, "/home/sebastiao_zoio/Documents/test_mewpy/mewpy/examples/models/germ/e_coli_core.xml")
    reader_time = time.process_time() - start
    start = time.process_time()
    read_model(gem_reader, trn_reader)
    read_model_time = time.process_time() - start
    return reader_time, read_model_time

In [10]:
times = []

for e in range(20):
    times.append(get_old_times())

times
reader_times = 0.0
read_model_times = 0.0
for t in times:
    reader_times += t[0]
    read_model_times += t[1]

n = len(times)
old_times_means = 0.0, reader_times/n, read_model_times/n
old_times_means

(0.0, 5.214170000051865e-05, 0.49308693479999877)

In [32]:
def get_fba_times(model):
    start = time.process_time()
    fba = RFBA(model)
    constructor_time = time.process_time() - start

    start = time.process_time()
    fba = fba.build()
    build_time = time.process_time() - start

    start = time.process_time()
    fba = fba.optimize()
    optimize_time = time.process_time() - start

    return constructor_time, build_time, optimize_time

In [33]:
times = [0.0, 0.0, 0.0]
for i in range(50):
    new_time = get_fba_times(germ_model)
    times[0] += new_time[0]
    times[1] += new_time[1]
    times[2] += new_time[2]

times[0]/50 ,times[1]/50, times[2]/50

(2.345351999565537e-05, 1.1136587577199988, 0.0716094681200002)

In [34]:
times = [0.0, 0.0, 0.0]
for i in range(50):
    new_time = get_fba_times(old_gem_model)
    times[0] += new_time[0]
    times[1] += new_time[1]
    times[2] += new_time[2]

times[0]/50 ,times[1]/50, times[2]/50

(2.901894000046923e-05, 0.020055404000002, 0.008102763099992671)

In [43]:
ttime= 0.0
n=50
for e in range(n):
    start = time.process_time()
    fva(germ_model)
    ttime += time.process_time() - start
ttime/n

0.11332804028000083

In [44]:
ttime= 0.0
n=50
for e in range(n):
    start = time.process_time()
    fva(old_gem_model)
    ttime += time.process_time() - start
ttime/n

0.16623355740000165